处理数据

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

test_sentence = []
txt = ""
with open('词性标注.txt','r',encoding='utf-8') as f:
    txt = f.read()
test_sentence = txt.split()
# 将单词序列转化为数据元组列表，
# 其中的每个元组格式为([ word_i-2, word_i-1 ], target word)
trigrams = [ ([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence) - 2) ]

# 打印出前3条数据，注意观察数据的结构
print(trigrams[:3])

[(['第一回/m', '甄士隐/nr'], '梦幻/n'), (['甄士隐/nr', '梦幻/n'], '识通/v'), (['梦幻/n', '识通/v'], '灵/n')]


准备词嵌入

In [4]:

# set 即去除重复的词
vocab = set(test_sentence)
# 建立词典，它比单词表多了每个词的索引
word_to_ix = { word: i for i, word in enumerate(vocab) }
print(len(vocab))


{'顺溜溜/z', '六十多年/m', '通在/d', '绳/n', '问柳评花/n', '黄莺儿/nr', '心实/n', '上命/v', '谋反/v', '妈妈/n', '因房/n', '一匹/m', '恨事/n', '偏见/v', '缉贼/n', '胱/x', '陶令/n', '那荣宁/nr', '爻动/n', '来赖/v', '娘长/n', '共总/n', '列苍/nr', '恍/vg', '索命/nz', '忏宿/v', '尘/ng', '舜尧/n', '亲蜜/n', '瞒/v', '放开/v', '司来/nr', '咬舌/v', '一心/d', '禀道/v', '水落石出/i', '攀/v', '鲔/g', '几步/m', '殃/vg', '轻/a', '临走时/b', '思嫁/v', '风雨无阻/i', '自身/r', '十六大/j', '目下/t', '几辆/m', '即景/n', '象鼻/v', '那事/r', '家/m', '秋夜/t', 'й/x', '走火/v', '无措/v', '中准/n', '誉/v', '太/d', '理屈词穷/i', '打鼓/v', '乖滑/a', '鸣锣/n', '目眩/v', '第十七回/m', '一二个/m', '不演/v', '不均/a', '兄妹/n', '靠你了/l', '冒失/v', '成画/n', '商议/n', '花费/n', '马上/d', '本房/n', '帕/nr', '几两/m', '公评/n', '阴人/n', '自泪/l', '鸳枕/n', '会衔/n', '海市/ns', '乱子/n', '困眼/a', '贾蔷/nr', '走漏风声/i', '二十/m', '父名/n', '访道/v', '凤头/n', '寅时/t', '香销/n', '亲王/n', '联道/v', '结实/n', '若论/c', '树头/n', '推让/v', '带断/v', '招出/v', '所叹者/n', '以至于/c', '早/a', '耐性/n', '玉带/n', '皇商/n', '鬼神/n', '如有所失/i', '这日/r', '欹/e', '当晚/t', '缓决/v', '贾珍献爵/nr', '赎/v', '假语/n', '派出/v', '联牌/v', '百病/n', '能动性/n', '姽/zg', 

建立模型

In [ ]:
# 上下文大小
# 即 前两个词
CONTEXT_SIZE = 2
# 嵌入维度
EMBEDDING_DIM = 1
context = test_sentence

In [ ]:
class NGramLanguageModeler(nn.Module):

    # 初始化时需要指定：单词表大小、想要嵌入的维度大小、上下文的长度
    def __init__(self, vocab_size, embedding_dim, context_size):
        # 继承自nn.Module，例行执行父类super 初始化方法
        super(NGramLanguageModeler, self).__init__()
        # 建立词嵌入模块
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 线性层1
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        # 线性层2，隐藏层 hidden_size 为128
        self.linear2 = nn.Linear(128, vocab_size)

    # 重写的网络正向传播方法
    # 只要正确定义了正向传播
    # PyTorch 可以自动进行反向传播
    def forward(self, inputs):
        # 将输入进行“嵌入”，并转化为“行向量”
        embeds = self.embeddings(inputs).view((1, -1))
        # 嵌入后的数据通过线性层1后，进行非线性函数 ReLU 的运算
        out = F.relu(self.linear1(embeds))
        # 通过线性层2后
        out = self.linear2(out)
        # 通过 log_softmax 方法将结果映射为概率的log
        # log 概率是用于下面计算负对数似然损失函数时方便使用的
        log_probs = F.log_softmax(out)
        return log_probs

In [ ]:
context_idxs = list(map(lambda w: word_to_ix[w], context))

开始训练

In [ ]:
losses = []
# 损失函数为 负对数似然损失函数(Negative Log Likelihood)
loss_function = nn.NLLLoss()
# 实例化我们的模型，传入：
# 单词表的大小、嵌入维度、上下文长度
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
# 优化函数使用随机梯度下降算法，学习率设置为0.001
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(1000):
    total_loss = 0
    # 循环context上下文，比如：['When', 'forty']
    # target，比如：winters
    for context, target in trigrams:

        # 步骤1：准备数据
        # 将context如“['When', 'forty']”
        # 转化为索引，如[68, 15]
        # 再建立为 PyTorch Variable 变量，以计算梯度
        context_idxs = list(map(lambda w: word_to_ix[w], context))
        context_var = autograd.Variable( torch.LongTensor(context_idxs) )

        # 步骤2：清空梯度值，防止上次的梯度累计
        model.zero_grad()

        # 步骤3：运行网络的正向传播，获得 log 概率
        log_probs = model(context_var)

        # 步骤4：计算损失函数
        # 传入 target Variable
        loss = loss_function(log_probs, autograd.Variable(torch.LongTensor([word_to_ix[target]])))

        # 步骤5：进行反向传播并更新梯度
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    losses.append(total_loss)

print('Finished')

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

plt.figure()
plt.plot(losses)